## Sentinel-2 L2A 

Read Sentinel-2 from Planetary Computer

In [ ]:
from dotenv import load_dotenv

load_dotenv()

import os

import fsspec
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import numpy as np
import odc.stac
import planetary_computer as pc
import pystac
import rioxarray
import shapely
import stac_geoparquet
import xarray as xr

### Dask client

In [ ]:
from distributed import Client

client = Client()
client

### Coastal grid

Fetch coastal grid with specified buffer size and zoom level. The buffer is derived from OSM coastline. Increasing the zoom level reduces the tile size (SlippyMapTiles). 

In [ ]:
from coastpy.utils.grid import coastal_grid_by_mgrs_tile, read_coastal_grid

grid = read_coastal_grid(buffer_size="10000m", zoom=9)
grid = coastal_grid_by_mgrs_tile(grid)

### Define region of interest

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = m.center = (53.19, 4.9)
m.zoom = 12
m.layout.height = "800px"
m

In [ ]:
from coastpy.geo.utils import get_region_of_interest_from_map

roi = get_region_of_interest_from_map(m, default_extent=(4.757, 53.108, 5.042, 53.272))

## Define input parameters

- The coastal zone shows how to mask by geometry
- ODC GeoBox model is used to reproject raster data onto a rectilinear grid (typically defined in meters, UTM).

In [ ]:
grid_roi = gpd.sjoin(grid, roi)

coastal_zone = odc.geo.geom.Geometry(roi.geometry.item(), crs=roi.crs)

try:
    utm_epsg = grid_roi["coastal_grid:utm_epsg"].unique().item()
except:
    print("This tutorial notebook only works for region of interests that are within one UTM zone.")

coastal_zone = odc.geo.geom.Geometry(roi.geometry.item(), crs=roi.crs)
geobox = odc.geo.geobox.GeoBox.from_geopolygon(
    coastal_zone.to_crs(utm_epsg), resolution=10
)
geobox

### Retrieve Sentinel-2 L2A

In [ ]:
from coastpy.eo.collection import S2Collection

s2 = (
    S2Collection(stac_cfg={})
    .search(
        roi,
        date_range="2023-06-01/2023-08-31",
        query={"eo:cloud_cover": {"lt": 20}},
    )
    .load(
        bands=["blue", "green", "red", "swir16", "SCL"],
        patch_url=pc.sign,
        chunks={},
        resampling="cubic",
        dtype="float32",
        resolution=10,
        crs="utm",
    )
    .mask_and_scale(
        mask_geometry=coastal_zone,
        mask_nodata=True,
        scale=True,
        scale_factor=0.0001,
        scale_vars_to_skip=["SCL"],
        mask_scl=["NO_DATA", "DARK_AREA_PIXELS", "CLOUDS_HIGH_PROBABILITY"],
    )
    .add_spectral_indices(["MNDWI"])
    .execute(compute=False)
)
s2

In [ ]:
xx = s2.compute()

## Compute Sentinel-2 L2A Composite

Same as above, but then using the composite method while filtering out the optimal imagery. 

In [ ]:
from datetime import timedelta

import pystac

from coastpy.eo.filter import filter_and_sort_stac_items


def filter_stac_items(items: list[pystac.Item]) -> list[pystac.Item]:
    """Filter and sort STAC items."""
    return filter_and_sort_stac_items(
        items,
        group_by=["s2:mgrs_tile", "sat:relative_orbit"],
        time_window=timedelta(days=5),
        max_num_groups=4,
        max_items=30,  
        only_summer=True,
        verbose=False,
    )

In [ ]:
from coastpy.eo.collection import S2Collection

composite = (
    S2Collection(stac_cfg={})
    .search(
        roi,
        date_range="2023-06-01/2023-08-31",
        query={"eo:cloud_cover": {"lt": 20}},
    )
    .load(
        bands=["blue", "green", "red", "swir16", "SCL"],
        patch_url=pc.sign,
        chunks={},
        resampling="cubic",
        dtype="float32",
        resolution=10,
        crs="utm",
    )
    .mask_and_scale(
        mask_geometry=coastal_zone,
        mask_nodata=True,
        scale=True,
        scale_factor=0.0001,
        scale_vars_to_skip=["SCL"],
        mask_scl=["NO_DATA", "DARK_AREA_PIXELS", "CLOUDS_HIGH_PROBABILITY"],
    )
    .composite(method="simple", percentile=50, filter_function=filter_stac_items)
    .add_spectral_indices(["MNDWI"])
    .execute(compute=False)
)
composite

In [ ]:
cc = composite.compute()

## Plot data on a map

- Pay attention to nodata value; this value depends on how the data is retrieved.

In [ ]:
import panel as pn

pn.Column(
    cc.MNDWI.where(lambda xx: xx != -9999).hvplot(
        x="x", y="y", rasterize=True, geo=True, tiles="OSM"
    )
    * roi[["geometry"]].hvplot(geo=True, line_color="red", fill_color=None)
).show()